In [ ]:
#!pip install geopandas
#!pip install contextily
#!pip list
#!nvcc --version
!nvidia-smi

Wed May 31 17:33:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Importaciones de bibliotecas estándar de Python
from datetime import datetime, timedelta

# Importaciones de bibliotecas de terceros
#import geopandas as gpd
import pandas as pd
import os as os
import numpy as np
import traceback
import os
from shapely.geometry import box
from shapely.ops import unary_union
import matplotlib.pyplot as plt
#import contextily as ctx
import pandas as pd
from pandas import Timestamp
import unicodedata
from PIL import Image
from IPython.display import display
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras import layers, Model
from tensorflow.python.keras.optimizers import adam_v2
from tensorflow.python.keras.metrics import Mean
from keras.losses import MeanSquaredError
from google.colab import drive

In [ ]:
tf.config.list_physical_devices('GPU')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
print(os.listdir('/content/drive/MyDrive/MGN2021_MPIO_POLITICO/weights/'))

FileNotFoundError: ignored

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
gdf=pd.read_file("/content/drive/MyDrive/MGN2021_MPIO_POLITICO/MGN_MPIO_POLITICO.shp")
gdf.head()

AttributeError: ignored

In [ ]:
def remove_accents_and_uppercase(text):
    if not isinstance(text, str):
        return text

    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    return text.upper()

In [ ]:
columnas_a_transformar_colombia = ["DPTO_CNMBR", "MPIO_CNMBR"]
gdf[columnas_a_transformar_colombia]=gdf[columnas_a_transformar_colombia].applymap(remove_accents_and_uppercase)
gdf.head()

,DPTO_CCDGO,MPIO_CCDGO,MPIO_CDPMP,DPTO_CNMBR,MPIO_CNMBR,MPIO_CRSLC,MPIO_NAREA,MPIO_CSMBL,MPIO_VGNC,MPIO_TIPO,Shape_Leng,Shape_Area,geometry
0,05,001,05001,ANTIOQUIA,MEDELLIN,1965,374.812877,4,2021,MUNICIPIO,1.032546,0.030606,"POLYGON ((-75.66974 6.37360, -75.66965 6.37360..."
1,05,002,05002,ANTIOQUIA,ABEJORRAL,1814,507.141095,4,2021,MUNICIPIO,1.158504,0.041384,"POLYGON ((-75.46938 5.94575, -75.46897 5.94571..."
2,05,004,05004,ANTIOQUIA,ABRIAQUI,1912,296.894050,4,2021,MUNICIPIO,0.812183,0.024248,"POLYGON ((-76.08351 6.75050, -76.08325 6.75048..."
3,05,021,05021,ANTIOQUIA,ALEJANDRIA,Decreto departamental 304 de 1907,128.932153,4,2021,MUNICIPIO,0.705200,0.010535,"POLYGON ((-75.03320 6.41586, -75.03313 6.41585..."
4,05,030,05030,ANTIOQUIA,AMAGA,1912,84.132675,4,2021,MUNICIPIO,0.445241,0.006867,"POLYGON ((-75.67587 6.08561, -75.67540 6.08491..."


In [ ]:
gdf = gdf.to_crs('EPSG:3857')
gdf['latitud'] = gdf['geometry'].centroid.y

NameError: ignored

In [ ]:
gdf_sorted = gdf.sort_values('latitud', ascending=False)

In [ ]:
gdf_sorted = gdf_sorted.reset_index(drop=True)

In [ ]:
gdf_sorted.head()

NameError: ignored

In [ ]:
len(gdf_sorted)

1121

In [ ]:
municipio_a_indice = {(row['DPTO_CNMBR'], row['MPIO_CNMBR']): i for i, row in gdf_sorted.iterrows()}
len(municipio_a_indice)

1121

In [ ]:
indice_a_municipio = {indice: municipio for municipio, indice in municipio_a_indice.items()}
municipios = [indice_a_municipio[i] for i in range(len(indice_a_municipio))]


In [ ]:
print(municipios)

[('ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'PROVIDENCIA'), ('ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'SAN ANDRES'), ('LA GUAJIRA', 'URIBIA'), ('LA GUAJIRA', 'MANAURE'), ('LA GUAJIRA', 'MAICAO'), ('LA GUAJIRA', 'RIOHACHA'), ('LA GUAJIRA', 'ALBANIA'), ('MAGDALENA', 'SANTA MARTA'), ('LA GUAJIRA', 'HATONUEVO'), ('LA GUAJIRA', 'DIBULLA'), ('ATLANTICO', 'PUERTO COLOMBIA'), ('LA GUAJIRA', 'BARRANCAS'), ('ATLANTICO', 'BARRANQUILLA'), ('LA GUAJIRA', 'DISTRACCION'), ('ATLANTICO', 'TUBARA'), ('ATLANTICO', 'SOLEDAD'), ('ATLANTICO', 'GALAPA'), ('MAGDALENA', 'SITIONUEVO'), ('MAGDALENA', 'CIENAGA'), ('ATLANTICO', 'MALAMBO'), ('MAGDALENA', 'PUEBLOVIEJO'), ('LA GUAJIRA', 'FONSECA'), ('ATLANTICO', 'JUAN DE ACOSTA'), ('LA GUAJIRA', 'SAN JUAN DEL CESAR'), ('ATLANTICO', 'SABANAGRANDE'), ('MAGDALENA', 'ZONA BANANERA'), ('ATLANTICO', 'BARANOA'), ('ATLANTICO', 'POLONUEVO'), ('ATLANTICO', 'SANTO TOMAS'), ('ATLANTICO', 'PIOJO'), ('ATLANTICO', 'USIACURI'), ('ATLANTICO', 'PAL

In [ ]:
print(municipio_a_indice)

{('ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'PROVIDENCIA'): 0, ('ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'SAN ANDRES'): 1, ('LA GUAJIRA', 'URIBIA'): 2, ('LA GUAJIRA', 'MANAURE'): 3, ('LA GUAJIRA', 'MAICAO'): 4, ('LA GUAJIRA', 'RIOHACHA'): 5, ('LA GUAJIRA', 'ALBANIA'): 6, ('MAGDALENA', 'SANTA MARTA'): 7, ('LA GUAJIRA', 'HATONUEVO'): 8, ('LA GUAJIRA', 'DIBULLA'): 9, ('ATLANTICO', 'PUERTO COLOMBIA'): 10, ('LA GUAJIRA', 'BARRANCAS'): 11, ('ATLANTICO', 'BARRANQUILLA'): 12, ('LA GUAJIRA', 'DISTRACCION'): 13, ('ATLANTICO', 'TUBARA'): 14, ('ATLANTICO', 'SOLEDAD'): 15, ('ATLANTICO', 'GALAPA'): 16, ('MAGDALENA', 'SITIONUEVO'): 17, ('MAGDALENA', 'CIENAGA'): 18, ('ATLANTICO', 'MALAMBO'): 19, ('MAGDALENA', 'PUEBLOVIEJO'): 20, ('LA GUAJIRA', 'FONSECA'): 21, ('ATLANTICO', 'JUAN DE ACOSTA'): 22, ('LA GUAJIRA', 'SAN JUAN DEL CESAR'): 23, ('ATLANTICO', 'SABANAGRANDE'): 24, ('MAGDALENA', 'ZONA BANANERA'): 25, ('ATLANTICO', 'BARANOA'): 26, ('ATLANTICO', 'POLONUEVO'): 2

Con la plantilla lista para ser usada, se procede a crear cada capa como una imagen distinta por cada dia desde que se registro el primer feminicidio en el dataframe del observatorio

In [ ]:
feminicidios = pd.read_csv('/content/drive/MyDrive/MGN2021_MPIO_POLITICO/datos_observatorio_limpios.csv', encoding='utf-8', sep=',')

feminicidios['num_casos'] = feminicidios['num_casos'].astype(int)

feminicidios['fecha_en_prensa'] = pd.to_datetime(feminicidios['fecha_en_prensa'])

feminicidios.head()

,fecha_en_prensa,departamento,municipio,num_casos,num_casos_normalizado
0,2018-01-02,ANTIOQUIA,HISPANIA,1,0.00
1,2018-01-02,META,VILLAVICENCIO,1,0.00
2,2018-01-03,ANTIOQUIA,MEDELLIN,3,0.50
3,2018-01-03,VALLE DEL CAUCA,CALI,2,0.25
4,2018-01-05,CORDOBA,MONTERIA,1,0.00


In [ ]:
fecha_mas_antigua = feminicidios['fecha_en_prensa'].min()
ultima_fecha = feminicidios['fecha_en_prensa'].max()

print(fecha_mas_antigua,'-',ultima_fecha)

2018-01-02 00:00:00 - 2023-01-28 00:00:00


In [ ]:
diferencia = ultima_fecha- fecha_mas_antigua
num_dias = diferencia.days
print(num_dias)
fecha_inicial=fecha_mas_antigua
fecha_final=ultima_fecha

1852


In [ ]:
min_num_casos = feminicidios['num_casos'].min()
max_num_casos = feminicidios['num_casos'].max()
feminicidios['num_casos_normalizado'] = (feminicidios['num_casos'] - min_num_casos) / (max_num_casos - min_num_casos)

In [ ]:
fecha_min = feminicidios['fecha_en_prensa'].min()
fecha_max = feminicidios['fecha_en_prensa'].max()
rango_fechas = pd.date_range(fecha_min, fecha_max)

In [ ]:
data = {fecha: [0]*len(municipio_a_indice) for fecha in rango_fechas}

In [ ]:
for _, row in feminicidios.iterrows():
    try:
        fecha = row['fecha_en_prensa']
        departamento = row['departamento']
        municipio = row['municipio']
        indice = municipio_a_indice[(departamento, municipio)]
        data[fecha][indice] = 1
    except KeyError:
        print(f"KeyError for departamento: {departamento}, municipio: {municipio}, fecha: {fecha}")


In [ ]:
print(municipio_a_indice)

{('ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'PROVIDENCIA'): 0, ('ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'SAN ANDRES'): 1, ('LA GUAJIRA', 'URIBIA'): 2, ('LA GUAJIRA', 'MANAURE'): 3, ('LA GUAJIRA', 'MAICAO'): 4, ('LA GUAJIRA', 'RIOHACHA'): 5, ('LA GUAJIRA', 'ALBANIA'): 6, ('MAGDALENA', 'SANTA MARTA'): 7, ('LA GUAJIRA', 'HATONUEVO'): 8, ('LA GUAJIRA', 'DIBULLA'): 9, ('ATLANTICO', 'PUERTO COLOMBIA'): 10, ('LA GUAJIRA', 'BARRANCAS'): 11, ('ATLANTICO', 'BARRANQUILLA'): 12, ('LA GUAJIRA', 'DISTRACCION'): 13, ('ATLANTICO', 'TUBARA'): 14, ('ATLANTICO', 'SOLEDAD'): 15, ('ATLANTICO', 'GALAPA'): 16, ('MAGDALENA', 'SITIONUEVO'): 17, ('MAGDALENA', 'CIENAGA'): 18, ('ATLANTICO', 'MALAMBO'): 19, ('MAGDALENA', 'PUEBLOVIEJO'): 20, ('LA GUAJIRA', 'FONSECA'): 21, ('ATLANTICO', 'JUAN DE ACOSTA'): 22, ('LA GUAJIRA', 'SAN JUAN DEL CESAR'): 23, ('ATLANTICO', 'SABANAGRANDE'): 24, ('MAGDALENA', 'ZONA BANANERA'): 25, ('ATLANTICO', 'BARANOA'): 26, ('ATLANTICO', 'POLONUEVO'): 2

In [ ]:
# Convertir el diccionario a una lista de tuplas (fecha, vector)
data_list = [(fecha, vector) for fecha, vector in data.items()]

# Ordenar la lista por fecha
data_list.sort(key=lambda x: x[0])

In [ ]:
def create_input_sequences(data, start_date, end_date, window_size):
    start_index = None
    end_index = None

    for i, (date, _) in enumerate(data):
        if date == start_date and start_index is None:
            start_index = i
        if date == end_date:
            end_index = i

    if start_index is None or end_index is None:
        raise ValueError('Start date or end date not found in data.')

    input_sequences = []
    dates = []

    for i in range(start_index + window_size, end_index + 1):
        sequence = [vector for _, vector in data[i-window_size:i]]
        sequence_date = data[i][0]
        input_sequences.append(sequence)
        dates.append(sequence_date)

    return input_sequences, dates


In [ ]:
# Define el rango de fechas
start_date = Timestamp('2022-01-01') - pd.Timedelta(days=42)  # Ajusta la fecha de inicio a 42 días antes
end_date = Timestamp('2022-01-02')

# Crear secuencias de entrada
input_sequences, dates = create_input_sequences(data_list, start_date, end_date, 42)

print('Dates:', dates)

Dates: [Timestamp('2022-01-01 00:00:00', freq='D'), Timestamp('2022-01-02 00:00:00', freq='D')]


In [ ]:
num_municipios=len(municipios)

In [ ]:
# Generador PRUEBA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
seq_length = 42
# Generador
def build_cgan_generator(latent_dim, n_cats):
    # Label Inputs
    in_label = layers.Input(shape=(1,))
    lbls = layers.Embedding(n_cats, 50)(in_label)
    lbls = layers.Dense((seq_length) * num_municipios)(lbls)
    lbls = layers.Reshape((seq_length, num_municipios))(lbls)  # Ajustamos las dimensiones

    # Generator Inputs (latent vector)
    in_latent = layers.Input(shape=(latent_dim,))
    gen = layers.Dense((seq_length ) * num_municipios)(in_latent)
    gen = layers.Reshape((seq_length, num_municipios))(gen)  # Ajustamos las dimensiones

    # Combine both inputs
    concat = layers.Concatenate(axis=-1)([gen, lbls])

    # Hidden Layer 1
    g = layers.Conv1D(filters=128, kernel_size=3, padding='same')(concat)
    g = layers.LeakyReLU(alpha=0.2)(g)

    # Hidden Layer 2
    g = layers.Conv1D(filters=128, kernel_size=3, padding='same')(g)
    g = layers.LeakyReLU(alpha=0.2)(g)

    # Output Layer
    output_layer = layers.Dense(num_municipios, activation='sigmoid')(g)

    # Define model
    model = Model([in_latent, in_label], output_layer)
    return model

# Discriminador
def build_cgan_discriminator(seq_length, num_municipios, n_cats):
    # Image input
    in_img = layers.Input(shape=(seq_length, num_municipios))

    # Label input
    in_label = layers.Input(shape=(1,))
    lbls = layers.Embedding(n_cats, 50)(in_label)
    lbls = layers.Dense((seq_length ) * num_municipios)(lbls)
    lbls = layers.Reshape((seq_length, num_municipios))(lbls)

    # Concatenate image and label inputs
    concat = layers.Concatenate(axis=-1)([in_img, lbls])

    # Hidden Layer 1
    d = layers.Conv1D(filters=128, kernel_size=3, padding='same')(concat)
    d = layers.LeakyReLU(alpha=0.2)(d)

    # Hidden Layer 2
    d = layers.Conv1D(filters=128, kernel_size=3, padding='same')(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Flatten()(d)

    # Output Layer
    output_layer = layers.Dense(1, activation='sigmoid')(d)

    # Define model
    model = Model([in_img, in_label], output_layer)
    return model




In [ ]:
#Modelo optimizado PRUEBA Urcuqui
class CGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(CGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(CGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_sequences):
        if isinstance(real_sequences, tuple):
            real_sequences = real_sequences[0]

        # Sample random points in the latent space
        batch_size = tf.shape(real_sequences)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake sequences
        generated_sequences = self.generator.fit(random_latent_vectors, steps_per_epoch=10)

        # Combine them with real sequences
        combined_sequences = tf.concat([generated_sequences, real_sequences], axis=0)

        # Assemble labels discriminating real from fake sequences
        labels = tf.concat(
            [tf.ones((batch_size, 1)) * 0.9, tf.zeros((batch_size, 1)) * 0.1], axis=0
        ) # Aplicamos el suavizado de etiquetas

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator.fit(combined_sequences, steps_per_epoch=10)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real sequences"
        misleading_labels = tf.ones((batch_size, 1)) * 0.9

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"d_loss": d_loss, "g_loss": g_loss}



In [ ]:
#Modelo CGAN Optimizado Inicial
class CGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(CGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(CGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_sequences):
        if isinstance(real_sequences, tuple):
            real_sequences = real_sequences[0]

        # Sample random points in the latent space
        batch_size = tf.shape(real_sequences)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake sequences
        generated_sequences = self.generator(random_latent_vectors)

        # Combine them with real sequences
        combined_sequences = tf.concat([generated_sequences, real_sequences], axis=0)

        # Assemble labels discriminating real from fake sequences
        labels = tf.concat(
            [tf.ones((batch_size, 1)) * 0.9, tf.zeros((batch_size, 1)) * 0.1], axis=0
        ) # Aplicamos el suavizado de etiquetas

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_sequences)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real sequences"
        misleading_labels = tf.ones((batch_size, 1)) * 0.9

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"d_loss": d_loss, "g_loss": g_loss}

In [ ]:
def map_municipios_to_indices(municipios, municipio_a_indice):
    return [municipio_a_indice[municipio] for municipio in municipios]

In [ ]:
def data_loader(input_sequences, municipios, batch_size, municipio_a_indice):
    while True:
        # Crear índices aleatorios para el batch
        idx = np.random.randint(0, len(input_sequences), batch_size)

        # Crear el batch de secuencias y municipios
        batch_seqs = np.array([input_sequences[i] for i in idx])
        batch_municipios = [municipios[i] for i in idx]

        # Mapear los municipios a sus índices correspondientes
        batch_labels = map_municipios_to_indices(batch_municipios, municipio_a_indice)

        yield batch_seqs, batch_labels


In [ ]:
# Parámetros Iniciales con el data Loader
seq_length = 42  # Longitud de las secuencias de tiempo
num_municipios = len(municipios)  # Longitud de los vectores en tus secuencias
latent_dim = 20
batch_size = 128
num_epochs = 100
n_cats = len(municipios)

# Crear modelos y optimizadores
generator = build_cgan_generator(latent_dim, n_cats)
discriminator = build_cgan_discriminator(seq_length, num_municipios, n_cats)

generator.summary()
discriminator.summary()

generator_optimizer = adam_v2.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = adam_v2.Adam(learning_rate=0.0002, beta_1=0.5)

# Crear dataloader
#data_loader = dataloader(input_sequences, batch_size)
data_gen = data_loader(input_sequences, municipios, batch_size, municipio_a_indice)



Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 50)        56050       input_41[0][0]                   
__________________________________________________________________________________________________
dense_51 (Dense)                (None, 11210)        235410      input_42[0][0]                   
___________________________________________________________________________________________

In [ ]:
# Ciclo de entrenamiento AYUDAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA EXAMPLE
# Define metrics
generator_loss_metric = tf.keras.metrics.Mean(name='generator_loss')
discriminator_loss_metric = tf.keras.metrics.Mean(name='discriminator_loss')

# Define lists to hold history of losses
generator_loss_history = []
discriminator_loss_history = []

# Antes de comenzar el entrenamiento, puedes cargar los pesos del modelo si existen.
try:
    generator.load_weights('/content/drive/MyDrive/MGN2021_MPIO_POLITICO/weights/generator_weights.h5')
    discriminator.load_weights('/content/drive/MyDrive/MGN2021_MPIO_POLITICO/weights/discriminator_weights.h5')
    print("Loaded weights from disk.")
except:
    print("No weights found. Training from scratch.")

# Ciclo de entrenamiento
for epoch in range(num_epochs):
    for i, (real_seqs, labels) in enumerate(data_gen):
        real_seqs = real_seqs[:,:seq_length // 4,:]
        labels = tf.reshape(labels, (-1, 1))

        # Actualiza el discriminador
        real_labels = tf.ones((real_seqs.shape[0], 1))
        fake_seqs = generator([tf.random.normal((real_seqs.shape[0], latent_dim)), labels])
        fake_labels = tf.zeros((real_seqs.shape[0], 1))

        with tf.GradientTape() as tape:
            real_preds = discriminator([real_seqs, labels])
            fake_preds = discriminator([fake_seqs, labels])

            real_loss = tf.keras.losses.binary_crossentropy(real_labels, real_preds)
            fake_loss = tf.keras.losses.binary_crossentropy(fake_labels, fake_preds)
            disc_loss = (real_loss + fake_loss) * 0.5

        discriminator_loss_metric.update_state(disc_loss)

        disc_grads = tape.gradient(disc_loss, discriminator.trainable_weights)
        discriminator_optimizer.apply_gradients(zip(disc_grads, discriminator.trainable_weights))

        # Actualiza el generador
        sampled_labels = tf.random.uniform((real_seqs.shape[0], 1), minval=0, maxval=n_cats, dtype=tf.int32)

        with tf.GradientTape() as tape:
            fake_seqs = generator([tf.random.normal((real_seqs.shape[0], latent_dim)), sampled_labels])
            fake_preds = discriminator([fake_seqs, sampled_labels])
            gen_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(fake_preds), fake_preds)

        generator_loss_metric.update_state(gen_loss)

        gen_grads = tape.gradient(gen_loss, generator.trainable_weights)
        generator_optimizer.apply_gradients(zip(gen_grads, generator.trainable_weights))

        # Guarda los pesos cada 50 pasos
        if i % 500 == 0:
            generator.save_weights('/content/drive/MyDrive/MGN2021_MPIO_POLITICO/weights/generator_weights.h5')
            discriminator.save_weights('/content/drive/MyDrive/MGN2021_MPIO_POLITICO/weights/discriminator_weights.h5')
            print(f"Epoch: {epoch}, Step: {i}, Gen Loss: {generator_loss_metric.result().numpy()}, Disc Loss: {discriminator_loss_metric.result().numpy()}")

        # Add current losses to history
        generator_loss_history.append(generator_loss_metric.result().numpy())
        discriminator_loss_history.append(discriminator_loss_metric.result().numpy())

        # Resetea las métricas al final de cada epoch
        generator_loss_metric.reset_states()
        discriminator_loss_metric.reset_states()


Loaded weights from disk.
